# DCGAN for Microstructure Generation on Micro2D Dataset

### Objective
To implement and evaluate a Deep Convolutional GAN (DCGAN) for generating synthetic microstructure images from the MICRO2D dataset

## Experimental Setup

### Dataset:

MICRO2D dataset:

87,379  2-phase microstructures

The Microstructures are periodic and 256x256 pixels

10 Classes - `['AngEllipse','GRF', 'NBSA', 'RandomEllipse', 'VoidSmall', 'VoidSmallBig',
'VoronoiLarge', 'VoronoiMedium', 'VoronoiMediumSpaced', 'VoronoiSmall']`
Model Configuration:

### Generator:

Input: 100-dimensional latent space (sampled from a Gaussian distribution).

Architecture:

Dense layer reshaped into a 16×16×256 tensor.

Progressive upsampling using Conv2DTranspose layers, culminating in 256×256×1 outputs.

Batch normalization layers to stabilize training dynamics.

LeakyReLU activation for hidden layers, with Tanh activation at the output layer.

### Discriminator:

Input: 256×256×1256×256×1 images.

Architecture:

Conv2D layers with progressively increasing filters (32→512).

Dropout layers for regularization.

Batch normalization to ensure stable updates.

Sigmoid activation for binary classification (real or fake).

### Training Parameters:

Latent dimension: 100

Batch size: 32

Learning rates:
Generator: 0.0001

Discriminator: 0.0002

Optimizer: Adam with β1=0.5,β2=0.5

### Evaluation Metrics:

Generator loss (Binary cross-entropy).

Discriminator loss (Binary cross-entropy).

Visual inspection of generated images.

## Procedure

### Data Preparation:

Images were loaded from the MICRO2D dataset using the h5py library.
Normalized pixel values to `[−1,1][−1,1].`
Added a channel dimension to ensure compatibility with TensorFlow models.

### Model Implementation:

Defined generator and discriminator models using the TensorFlow Keras API.
Compiled the DCGAN model with Binary Crossentropy loss.

### Training:

Implemented label smoothing for real images to improve discriminator performance.

Trained the discriminator and generator alternately within each step:

Discriminator: Minimized loss by classifying real images as 0.9 (smoothing) and fake images as 0.0(with noise for robustness).

Generator: Minimized loss by fooling the discriminator into classifying fake images as 1.0

Monitored generated images at regular intervals using a custom callback.

### Visualization:

Generated synthetic images after each epoch for qualitative analysis.
Displayed 5×5 grids of generated samples.

## Observations

The Deep Convolutional GAN (DCGAN) has challenges in generating high-quality and realistic images especially when tackling complex tasks such as microstructure generation. DCGAN frequently yields fuzzy and lower resolution images, this can result in gradients disappearing and hinder the generator from capturing intricate details. Additionally, DCGAN is prone to mode collapse,
limiting the generator to produce a limited set of images thus reducing diversity. Moreover,the model struggles to maintain consistency, in more complex image classes.

In [ ]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
file_1 = './MICRO2D_homogenized.h5'

In [ ]:
dataset_1 = h5py.File(file_1,'r')
print(list(dataset_1.keys()))

In [ ]:
def load_NBSA_images(file_path):
    with h5py.File(file_path, 'r') as dataset:
        if 'NBSA' in dataset:
            group = dataset['NBSA']
            keys = list(group.keys())
            if len(keys) > 0:
                dataset_name = keys[0]
                images = group[dataset_name][:]
                return images
            else:
                print("No dataset found in the 'NBSA' group.")
        else:
            print("'NBSA' class not found in the dataset.")
    return None
def normalize_images(images):
    images = images.astype('float32')
    images = images * 2 - 1
    return images 
def plot_images(images, class_name):
    num_images = min(10, images.shape[0])
    fig, axes = plt.subplots(1, num_images, figsize=(20, 4))
    fig.suptitle(f"Images for Class: {class_name}", fontsize=16)
    for i in range(num_images):
        axes[i].imshow(images[i], cmap='gray')
        axes[i].axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
NBSA = load_NBSA_images(file_1)

In [ ]:
plot_images(NBSA,'NBSA')

In [ ]:
NBSA.shape,NBSA.dtype

In [ ]:
train_images = normalize_images(NBSA)
train_images.shape,train_images.dtype

In [ ]:
train_images.min(),train_images.max()

In [ ]:
LATENT_DIM=100
image_size = 256   
channels = 1
IMAGE_SIZE = 256
CHANNELS = 1
Z_DIM = 100
LEARNING_RATE_D = 0.0002
LEARNING_RATE_G = 0.0001
ADAM_BETA_1 = 0.5
ADAM_BETA_2 = 0.5
NOISE_PARAM = 1 
EPOCHS = 500
BATCH_SIZE = 32

In [ ]:
import tensorflow as tf
from tensorflow.keras import (
    layers,
    models,
    callbacks,
    losses,
    utils,
    metrics,
    optimizers,
)
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img,array_to_img
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(len(train_images)).batch(BATCH_SIZE)

In [ ]:
def create_generator(LATENT_DIM):
    generator_input = layers.Input(shape=(LATENT_DIM,))
    x = layers.Dense(16 * 16 * 256, use_bias=False)(generator_input)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Reshape((16, 16, 256))(x)
    # First upsampling block
    x = layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    # Second upsampling block
    x = layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    # Second upsampling block
    x = layers.Conv2DTranspose(32, kernel_size=4, strides=2, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    # Third upsampling block to reach 256x256
    x = layers.Conv2DTranspose(1, kernel_size=4, strides=2, padding="same", use_bias=False, activation="tanh")(x)
    generator = models.Model(generator_input, x, name="Generator")
    return generator
generator = create_generator(LATENT_DIM)
generator.summary()

In [ ]:
def create_discriminator(image_size, channels):
    discriminator_input = layers.Input(shape=(image_size, image_size, channels))
    x = layers.Conv2D(32, kernel_size=4, strides=2, padding="same", use_bias=False)(discriminator_input)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Conv2D(64, kernel_size=4, strides=2, padding="same", use_bias=False)(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Conv2D(128, kernel_size=4, strides=2, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization(momentum=0.9)(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Conv2D(256, kernel_size=4, strides=2, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization(momentum=0.9)(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Conv2D(512, kernel_size=4, strides=2, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization(momentum=0.9)(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Flatten()(x)
    discriminator_output = layers.Dense(1, activation="sigmoid")(x)
    discriminator = models.Model(discriminator_input, discriminator_output, name="Discriminator")
    return discriminator
discriminator = create_discriminator(image_size, channels)
discriminator.summary()

In [ ]:
class DCGAN(Model):
    def __init__(self, generator, discriminator, latent_dim):
        super(DCGAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.latent_dim = latent_dim
        self.g_loss = Mean(name='g_loss')
        self.d_loss = Mean(name='d_loss')

    @property
    def metrics(self):
        return [self.g_loss, self.d_loss]

    def compile(self, g_optimizer, d_optimizer, loss_fn):
        super(DCGAN, self).compile()
        self.g_optimizer = g_optimizer
        self.d_optimizer = d_optimizer
        self.loss_fn = loss_fn

    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        random_noise = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Train Discriminator
        with tf.GradientTape() as tape:
            pred_real = self.discriminator(real_images, training=True)
            real_labels = tf.ones_like(pred_real) * 0.9  # Label smoothing for real labels
            d_loss_real = self.loss_fn(real_labels, pred_real)

            fake_images = self.generator(random_noise, training=False)
            pred_fake = self.discriminator(fake_images, training=True)
            fake_labels = tf.zeros_like(pred_fake) + 0.05 * tf.random.uniform(tf.shape(pred_fake))  # Noisy fake labels
            d_loss_fake = self.loss_fn(fake_labels, pred_fake)

            d_loss = (d_loss_real + d_loss_fake) / 2

        d_gradients = tape.gradient(d_loss, self.discriminator.trainable_variables)
        self.d_optimizer.apply_gradients(zip(d_gradients, self.discriminator.trainable_variables))

        # Train Generator
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_noise, training=True)
            pred_fake = self.discriminator(fake_images, training=True)
            labels = tf.ones_like(pred_fake)  # Generator tries to get fake images labeled as real
            g_loss = self.loss_fn(labels, pred_fake)

        g_gradients = tape.gradient(g_loss, self.generator.trainable_variables)
        self.g_optimizer.apply_gradients(zip(g_gradients, self.generator.trainable_variables))

        # Update metrics
        self.d_loss.update_state(d_loss)
        self.g_loss.update_state(g_loss)

        return {'d_loss': self.d_loss.result(), 'g_loss': self.g_loss.result()}


In [ ]:
class DCGANMonitor(callbacks.Callback):
    def __init__(self, num_imgs=25, latent_dim=100):
        self.num_imgs = num_imgs
        self.latent_dim = latent_dim
        self.noise = tf.random.normal([num_imgs, latent_dim])

    def on_epoch_end(self, epoch, logs=None):
        g_img = self.model.generator(self.noise, training=False)
        g_img = (g_img * 127.5) + 127.5
        g_img = tf.clip_by_value(g_img, 0, 255)
        g_img = g_img.numpy().astype("uint8")
        fig = plt.figure(figsize=(8, 8))
        for i in range(self.num_imgs):
            plt.subplot(5, 5, i + 1)
            if g_img.shape[-1] == 1:  
                plt.imshow(g_img[i, :, :, 0], cmap="gray")
            else:  
                plt.imshow(g_img[i])
            plt.axis("off")
        plt.show()

In [ ]:
dcgan = DCGAN(generator,discriminator,latent_dim=LATENT_DIM)

In [ ]:
dcgan.compile(g_optimizer=Adam(learning_rate=LEARNING_RATE_G,beta_1=ADAM_BETA_1),d_optimizer=Adam(learning_rate=LEARNING_RATE_D,beta_1=ADAM_BETA_2),loss_fn=BinaryCrossentropy())

In [ ]:
hist = dcgan.fit(train_dataset,epochs=1000,callbacks = [DCGANMonitor()])

In [ ]:
plt.suptitle('Loss')
plt.plot(hist.history['c_loss'],label = 'd_loss')
plt.plot(hist.history['g_loss'],label='g_loss')
plt.plot(hist.history['c_wass_loss'], label='w_loss')
plt.legend()
plt.show()

In [ ]:
def generate_and_plot_samples(generator, num_samples=100):
    noise = tf.random.normal([num_samples, 256, 256, 1])
    generated_images = generator(noise)
    generated_images = (generated_images + 1) / 2  # Denormalize
    
    fig, axs = plt.subplots(10, 10, figsize=(20, 20))
    for i, ax in enumerate(axs.flat):
        # Assuming the output is a 1D array of size 25
        img = generated_images[i].numpy().squeeze()
        img = img.reshape((5, 5))  # Reshape to 5x5
        ax.imshow(img, cmap='gray')
        ax.axis('off')
    plt.savefig('final_generated_samples.png')
    plt.close()

# Call the function
generate_and_plot_samples(generator)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from scipy.stats import entropy
from scipy import linalg

In [ ]:
def grayscale_to_rgb(images):
    return tf.image.grayscale_to_rgb(images)

def inception_score(images, model, batch_size=50, splits=10, epsilon=1e-16):
    preds = model.predict(images, batch_size=BATCH_SIZE)
    scores = []
    for i in range(splits):
        part = preds[i * (len(preds) // splits): (i + 1) * (len(preds) // splits), :]
        py = np.mean(part, axis=0)
        scores.append([])
        for j in range(part.shape[0]):
            pyx = part[j, :]
            scores[-1].append(entropy(pyx, py))
    
    scores = np.exp(np.mean(scores))
    return np.mean(scores), np.std(scores)

# Load pre-trained Inception model
inception_model = InceptionV3(include_top=True, weights='imagenet', input_shape=(299, 299, 3))

# Generate images and prepare them for Inception model
latent_dim = LATENT_DIM  # Make sure this matches your generator's input dimension
generated_images = generate_images(generator, LATENT_DIM, num_samples=1000)
generated_images = tf.image.resize(generated_images, (299, 299))
generated_images = grayscale_to_rgb(generated_images)  # Convert to RGB
generated_images = preprocess_input(generated_images * 255)  # Scale to [0, 255] and preprocess

# Calculate Inception Score
is_mean, is_std = inception_score(generated_images, inception_model)
print(f"Inception Score: {is_mean} ± {is_std}")

In [ ]:
def preprocess_images(images):
    # Resize images to 299x299
    images = tf.image.resize(images, (299, 299))
    # Convert grayscale to RGB if necessary
    if images.shape[-1] == 1:
        images = tf.image.grayscale_to_rgb(images)
    # Preprocess for InceptionV3
    images = preprocess_input(images * 255.0)
    return images

def calculate_fid(train_images, generated_images, model):
    def get_features(images):
        features = model.predict(images)
        return features

    # Preprocess both real and generated images
    real_images = preprocess_images(train_images)
    generated_images = preprocess_images(generated_images)

    real_features = get_features(real_images)
    gen_features = get_features(generated_images)
    
    mu1, sigma1 = np.mean(real_features, axis=0), np.cov(real_features, rowvar=False)
    mu2, sigma2 = np.mean(gen_features, axis=0), np.cov(gen_features, rowvar=False)
    
    ssdiff = np.sum((mu1 - mu2)**2.0)
    covmean = linalg.sqrtm(sigma1.dot(sigma2))
    
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

# Load pre-trained Inception model
inception_model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))

# Generate images
latent_dim = LATENT_DIM  
num_samples = 1000  
generated_images = generate_images(generator, LATENT_DIM, num_samples=num_samples)

# Calculate FID
fid = calculate_fid(train_images[:num_samples], generated_images, inception_model)
print(f"Fréchet Inception Distance: {fid}")